# Run apAdm
Goal:
Run qp-Admix for identifying question about Sardinian Demography.
Use a wrapper to run it with various parameters, for various populations.

Use two different set of outgroups: m13 and anc_euro. Save them in different subfolders.

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
# Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-e-16-229.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 28
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


# Define Some Standard Outgroups

In [2]:
### Parameters for qpAdm_run are set there!
a12 = ["Mota", "Ust_Ishim", "Kostenki14", "GoyetQ116-1", "Vestonice16", "MA1",
           "ElMiron", "Villabruna", "EHG", "CHG", "Natufian",
           "Levant_N"] # List of ancient European populations for right populations
distal = ["Anatolia_N", "WHG", "Steppe_EMBA", 
          "Iran_N", "Morocco_EN"]
print("# Anc_Euro: %i populations" % len(a12))
print("# Distal sources: %i populations" % len(distal))

# Anc_Euro: 12 populations
# Distal sources: 5 populations


In [3]:
def qpAdm_run(leftpops, rightpops, output_file, 
              input_folder = "./eigenstrat/combined/", 
              input_file="punic0.v43.geno",
              par_file_folder = "./parfiles/", 
              input_ind_suff="_mod1", 
              output_folder ="./output/qpAdm/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=False):
    """Run qpAdm. Write temporary parfile and run the analysis
    leftpops: List of left populations
    rightpops: List of right populations
    input_folder: Where to find the input files
    par_file_folder: Folder of the parameterfile
    input_file: The name of the input file
    input_ind_suff: Suffix of .ind file. To allow modified populations
    input_ind: name of the input .ind file. If given - write over default"""
    
    parfile_path = par_file_folder + "parfile_qpAdm"
    left_path, right_path = par_file_folder + "left", par_file_folder + "right"
    
    ### Create the parfile:
    with open(parfile_path, 'w') as f:
        f.write("%s\n" % ("DIR: " + input_folder))
        f.write("%s\n" % ("S1: " + input_file))
        
        indline = "indivname: DIR/S1" + input_ind_suff + ".ind"
        f.write("%s\n" % indline)
        f.write("%s\n" % "snpname: DIR/S1.snp")
        f.write("%s\n" % "genotypename: DIR/S1.geno")
        f.write("%s\n" % ("popleft: " + left_path))
        f.write("%s\n" % ("popright: " + right_path))
        f.write("%s\n" % "details: YES")   
        if all_snps==True:
            f.write("%s\n" % "allsnps: YES")
    
    ### Write leftpops rightpops:       
    with open(left_path, 'w') as f:
        f.write("\n".join(leftpops))
        
    with open(right_path, 'w') as f:
        f.write("\n".join(rightpops))
      
    ### Run qpAdm
    start = time()
    output_path = output_folder + output_file
    !$path_bin_qpAdm -p $parfile_path > $output_path
    end = time()
    print("Runtime: %2f" % (end - start))
    return 0


def load_iids_from_indfile(path_ind, string, 
                           col="clst", col_iid="iid"):
    """Load IIDs from Ind File
    Return List of IIDs"""
    df_ind = pd.read_csv(path_ind, delim_whitespace=True, header=None)
    df_ind.columns=["iid", "sex","clst"]
    idx = df_ind[col].str.contains(string)
    ls = df_ind[idx][col_iid].values
    return ls

### Test distal Modelling Run for single Individual

In [23]:
%%time
#iid = "I12433"  # The Iron Age African
#iid = "I11896"  # The old African one
#iid = "I18400"   # Steppe 1  # The Sardinian Outlier
# Iron Age Nigeria iid = "I15940"
#iid = "I18399"
iid = "I12847"


#sources = ["Anatolia_N", "WHG", "Steppe_EMBA", "Israel_MLBA_Canaanite", "Morocco_EN"]
sources = ["Anatolia_N", "WHG", "Steppe_EMBA", "Israel_MLBA_Canaanite", "Morocco_EN"]
#"Steppe_EMBA"  "WHG"

add_out = []
leftpops = [iid] + sources

qpAdm_run(leftpops = leftpops, 
          rightpops = a12+add_out, 
          output_file = ".".join(leftpops) + ".log", 
          input_folder = "./eigenstrat/combined/", 
          input_file = "punic1.v43",
          par_file_folder = "./parfiles/", 
          input_ind_suff = "_mod1", 
          output_folder = "./output/qpAdm/diverse/", 
          path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
          all_snps=True)

Runtime: 54.823291
CPU times: user 1.08 s, sys: 136 ms, total: 1.22 s
Wall time: 54.8 s


0

In [24]:
print("Finished run!")

Finished run!


### Run several Individuals

In [ ]:
### Load the individual IIDs

pops = ["Sardinia", "Algeria_N", "Punic"]
path_ind = "./eigenstrat/combined/punic0.v43.ind"

all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]

all_iids = np.concatenate(all_iids)
sources = distal

for iid in all_iids:
    qpAdm_run(leftpops = [iid] + sources, 
          rightpops = a12, 
          output_file = iid + "." + str(len(sources)) + "way.log", 
          input_folder = "./eigenstrat/combined/", 
          input_file = "punic0.v43",
          par_file_folder = "./parfiles/", 
          input_ind_suff = "_mod", 
          output_folder = "./output/qpAdm/5wayAll/", 
          path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
          all_snps=True)

Runtime: 64.200304
Runtime: 65.710443
Runtime: 63.501141
Runtime: 64.098155


## Run 2/3-way Models for African - European Admixture

In [6]:
### Load the individual IIDs
pops = ["Iberia_North_BA_Africa_all", "Algeria_N", 
        "Morocco_LN", "Punic_oAfrican", "Sardinia"]

path_ind = "./eigenstrat/combined/punic1.v43.ind"

all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)
#all_iids = ["I4246"] #The Iberian Outlier

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Anatolia_N", "WHG", "Morocco_EN"]
    add_out = ["Iran_N"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12+add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic0.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/3wayNAf/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

Doing Individual: MSR002
Runtime: 32.329889
Doing Individual: MSR003
Runtime: 32.197911
Doing Individual: ORC002
Runtime: 32.199739
Doing Individual: VIL004
Runtime: 32.198779
Doing Individual: VIL006
Runtime: 32.501195
Doing Individual: VIL007
Runtime: 32.166153
Doing Individual: VIL009
Runtime: 32.191764
Doing Individual: VIL010
Runtime: 32.105941
Doing Individual: VIL011
Runtime: 32.232729


# Run Models with Israle_MLBA_Canaaninte as Outgroup
Add as proxy for North African ancestry:
Morocco_EN, Algeria_N (I12433) or Iberian Outlier (I4246)

In [11]:
### Load the individual IIDs
pops = ["Iberia_Punic", "Sardinia", "Iberia_North_BA_Africa_all",
        "Morocco_LN"]
pops = ["Sicily_Phoenician"]

path_ind = "./eigenstrat/combined/punic1.v43.ind"

all_iids = [load_iids_from_indfile(path_ind=path_ind, 
                        string=pop) for pop in pops]
all_iids = np.concatenate(all_iids)
#all_iids = ["I7253"] # Do a single Individual

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Anatolia_N", "WHG", "Steppe_EMBA", "I12433"]
    add_out = ["Israel_MLBA_Canaanite"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic1.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/4way_p_Canaanite/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

Doing Individual: I7253
fatalx:
No samples: I7253
/usr/bin/sh: line 1: 11261 Aborted                 /n/groups/reich/hringbauer/o2bin/qpAdm -p ./parfiles/parfile_qpAdm > ./output/qpAdm/4way_p_Canaanite/I7253.Anatolia_N.WHG.Steppe_EMBA.I12433.log
Runtime: 22.123490


# Test Continutity to Mycenean Phoenician Group

In [ ]:
### Load the individual IIDs

all_iids = ["I18199", "I18200", "I18201", "I18203", "I18202", "I18189", "I18187"]

for iid in all_iids:
    print(f"Doing Individual: {iid}")
    sources = ["Mycenaean"]
    add_out = ["Israel_MLBA_Canaanite", "I12433"]
    leftpops = [iid] + sources

    qpAdm_run(leftpops = leftpops, 
              rightpops = a12 + add_out, 
              output_file = ".".join(leftpops) + ".log", 
              input_folder = "./eigenstrat/combined/", 
              input_file = "punic1.v43",
              par_file_folder = "./parfiles/", 
              input_ind_suff = "_mod1", 
              output_folder = "./output/qpAdm/t774_cont/", 
              path_bin_qpAdm = "/n/groups/reich/hringbauer/o2bin/qpAdm",
              all_snps=True)

# Explore the Ind File

In [4]:
df_ind = pd.read_csv("./eigenstrat/combined/punic0.v43.ind", delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex","clst"]

In [8]:
df_ind["clst"].value_counts()[:50]

Europe_LNBA                                   78
Europe_EN                                     29
Europe_MNChL                                  28
Anatolia_N                                    26
Israel_MLBA_Canaanite                         26
Steppe_EMBA                                   26
Steppe_MLBA                                   21
Italy_Sicily_Phoenician                       19
Italy_Sicily_IA_Polizzello                    19
England_ancient                               18
Levant_N                                      13
Italy_Sardinia_IA_Punic                        9
Iran_N                                         9
Armenia_MLBA                                   9
Morocco_Iberomaurusian                         6
SHG                                            6
Natufian                                       6
Armenia_ChL                                    5
Iberia_Hellenistic                             5
Minoan_Lasithi                                 5
Minoan_Odigitria    

In [7]:
df_ind[df_ind["clst"].str.contains("Myc")]

,iid,sex,clst
202,I9010,F,Mycenaean
204,I9006,F,Mycenaean
205,I9033,F,Mycenaean
206,I9041,M,Mycenaean


In [7]:
df_ind = pd.read_csv("./eigenstrat/combined/punic0.v43_mod.ind", delim_whitespace=True, header=None)
df_ind.columns=["iid", "sex","clst"]

In [ ]:
df_ind[df_ind["clst"].str.contains("Sardinia")]